# Shared response modeling
We project fMRI responses from each subject (20 subjects in total) into a shared low-dimensional space across 180 brain regions.
- [The shared response model](https://papers.nips.cc/paper/5855-a-reduced-dimension-fmri-shared-response-model) aims to learn this shared feature space.
- SRM can be summarized graphically from [Cohen et al., 2017](https://www.nature.com/articles/nn.4499)


## 1. mask epi data

### set mask epi data env

In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import copy

import numpy as np
from scipy import stats
import scipy.spatial.distance as sp_distance
from sklearn.svm import NuSVC

from brainiak.isc import isc
from brainiak.fcma.util import compute_correlation
import brainiak.funcalign.srm
from brainiak import image, io
import brainiak

import nibabel as nib
import nilearn as nil
import glob
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.image import concat_imgs, index_img

import matplotlib.pyplot as plt
from einops import rearrange

%autosave 5
%matplotlib inline

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/igeo_process"
fMRI_data_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_data/spatialfMRI_nii"

preprocessed_dir=project_dir+"/preprocess/preprocessed/afni_2023"
processed_dir=project_dir+"/process/processed"

num_runs = 14
num_TRs = 223
num_midlayer_units = 8
num_layer_pca_components = 20
num_deeplayers_units = num_midlayer_units + 12*num_layer_pca_components
num_semantic_categories = 10

start_fixation_TRs = 3
hemodynamic_shift_TRs = 3
num_TRs_video = 200

### load atlas brain regions

In [12]:
import os
# read design matrix
atlas_labels_txt = "Atlas MNI_Glasser_HCP_v1.0, 360 regions.txt"
atlas_labels_file = os.path.join(project_dir, "process/depth/mni_glasser_hcp", atlas_labels_txt)

in_file = open(atlas_labels_file,'r')
brain_region_dict={}

# distinguish left and right brain regions
atlas_all_lines = in_file.readlines()

for i_line in range(0,len(atlas_all_lines)):
    if "u:L_" in atlas_all_lines[i_line]:
        
        atlas_brain_region_label = []

        # left brain regions
        atlas_line_left = atlas_all_lines[i_line]
        atlas_temp = atlas_line_left.split(":")
        atlas_brain_region_name = atlas_temp[1][2:]
        # print(atlas_brain_region_name)
        atlas_brain_region_label_left = int(atlas_temp[2][:-1])
        atlas_brain_region_label.append(atlas_brain_region_label_left)
        
        # right brain regions
        atlas_line_right = atlas_all_lines[i_line+1]
        atlas_temp = atlas_line_right.split(":")
        atlas_brain_region_name = atlas_temp[1][2:]
        atlas_brain_region_label_right = int(atlas_temp[2][:-1])
        atlas_brain_region_label.append(atlas_brain_region_label_right)

        # put into dict
        brain_region_dict[atlas_brain_region_name] = atlas_brain_region_label

# print(len(brain_region_dict),brain_region_dict)


In [13]:
brain_region_name_list = list(brain_region_dict.keys())
brain_region_label_list = list(brain_region_dict.values())

folder_list = ["sub-01", "sub-02", "sub-03", "sub-04", "sub-05", 
               "sub-06", "sub-07", "sub-08", "sub-09", "sub-10", 
               "sub-11", "sub-12", "sub-13", "sub-14", "sub-15",
               "sub-16", "sub-17", "sub-18", "sub-19", "sub-20",
               ]

num_subs = len(folder_list)

### create region-based masks

In [ ]:
for i_brain_region_name in np.arange(0, len(brain_region_name_list)):
  
  brain_region_name = brain_region_name_list[i_brain_region_name]

  for sub_name in folder_list:

    mask_name = sub_name+"_MNI_Glasser_HCP_neural_mask.nii.gz"
    mask_file = os.path.join(processed_dir, sub_name, "shared_regions", mask_name)

    # Load the mask image
    mask = nib.load(mask_file)
    mask_data = mask.get_data()

    if i_brain_region_name != 0: # remove the effect of first region V1, id V1 is same as mask 0
      mask_data[mask_data==1] = 0

    for i_num_region in brain_region_label_list[i_brain_region_name]:
        mask_data[mask_data==i_num_region] = 1

    mask_data[mask_data!=1] = 0

    voxel_num = np.count_nonzero(mask_data[mask_data!=0])
    # print(voxel_num)

    mask_nifti = nib.Nifti1Image(mask_data, mask.affine, mask.header)

    saved_mask_name = sub_name + "_" + brain_region_name+"_MNI_Glasser_HCP_neural_mask.nii.gz"

    mask_path = os.path.join(processed_dir, sub_name, "shared_regions", saved_mask_name)

    nib.save(mask_nifti, mask_path)


### mask the epi data

In [ ]:
sub_masked_epi_data_list = []

for sub_name in folder_list:
  
  # print('sub name:',sub_name)

  brain_region_epi_data_list = []

  for brain_region_name in brain_region_name_list:

    print('sub name:',sub_name,',brain_region_name:',brain_region_name)

    ### load mask
    mask_name = sub_name + "_" + brain_region_name+"_MNI_Glasser_HCP_neural_mask.nii.gz"
    mask_file = os.path.join(processed_dir, sub_name, "shared_regions", mask_name)

    # Load the mask image
    mask = nib.load(mask_file)

    mask_data = mask.get_data()
    
    voxel_num = np.count_nonzero(mask_data[mask_data!=0])
    # mask = nib.Nifti1Image(mask_data, affine=np.eye(4))

    sub_masked_data = np.empty((voxel_num,0), float)

    sub_dir = os.path.join(processed_dir, sub_name)

    for run in range(1, num_runs+1):

      # load epi data
      current_file_str = 'pb04.{subject}.r{run:02}.combine+demean.nii.gz'
      current_wildcard_file_name = os.path.join(sub_dir, "shared_regions", current_file_str.format(subject=sub_name,run=run))

      current_file_name = glob.glob(current_wildcard_file_name)[0]
      epi_data = nib.load(current_file_name)

      epi_data = index_img(epi_data, slice(0, num_TRs))

      # print(sub_name,"run", run, epi_data.shape)

      # mask bold signal
      nifti_masker = NiftiMasker(mask_img=mask)
      masked_data = nifti_masker.fit_transform(epi_data)
      masked_data = masked_data.T

      # do it for epi data
      masked_data = stats.zscore(masked_data, axis=1, ddof=1)
      masked_data = np.nan_to_num(masked_data)
      # print("masked_data shape (voxel_num,TR_num)", masked_data.shape)

      sub_masked_data = np.append(sub_masked_data, masked_data, axis=1)

    # add a brain region
    brain_region_epi_data_list.append(sub_masked_data)

  # add a subject
  sub_masked_epi_data_list.append(brain_region_epi_data_list)


In [ ]:
print("number of subjects:", len(sub_masked_epi_data_list))
print("num of brain regions:", len(sub_masked_epi_data_list[0]))

for i in range(0, len(brain_region_name_list)):
    print(folder_list[0],',',brain_region_name_list[i],':',sub_masked_epi_data_list[0][i].shape)


### save and load sub_masked_epi_data_list

#### save

In [ ]:
import hdf5storage

matdic = {"sub_masked_epi_data_list":sub_masked_epi_data_list}
file_dir = processed_dir+"/depth/temp_data/"
hdf5storage.savemat(file_dir+"sub_masked_epi_data_list.mat", matdic)

#### load

In [ ]:
import hdf5storage

file_dir = processed_dir+"/depth/temp_data/"
matdic = hdf5storage.loadmat(file_dir+"sub_masked_epi_data_list.mat", )

sub_masked_epi_data_list = matdic["sub_masked_epi_data_list"]


In [ ]:
print("number of subjects:", len(sub_masked_epi_data_list))
print("num of brain regions:", len(sub_masked_epi_data_list[0]))

for i in range(0, len(brain_region_name_list)):
    print(folder_list[0],',',brain_region_name_list[i],':',sub_masked_epi_data_list[0][i].shape)


## 2. set feature dimensions

### set feature dimensions by The Optimal Hard Threshold

The optimal hard threshold for singular values is 4 / sqrt(3)

1.https://github.com/brainiak/brainiak/blob/ee093597c6c11597b0a59e95b48d2118e40394a5/brainiak/reprsimil/brsa.py#L157
2.http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6846297

#### calculate averaged feature dimensions by The Optimal Hard Threshold


In [ ]:
from brainiak.reprsimil.brsa import Ncomp_SVHT_MG_DLD_approx

regions_average_dims_list  = []

for i_region in range(0, len(brain_region_name_list)):

  region_dims_list = []

  for i_sub in range(0, num_subs):

    temp_masked_epi_data = sub_masked_epi_data_list[i_sub][i_region]
    ncomp = Ncomp_SVHT_MG_DLD_approx(temp_masked_epi_data.T) # 2-D numpy array of size [n_T, n_V]
    region_dims_list.append(ncomp)

  average_dims = np.mean(np.asarray(region_dims_list))
  regions_average_dims_list.append(np.floor(average_dims).astype(int))

regions_average_dims_array = np.asarray(regions_average_dims_list)
print("regions_average_dims_array:",regions_average_dims_array)

print("features in total:", np.sum(regions_average_dims_array))
plt.hist(regions_average_dims_array, bins=20)
plt.show()

#### iterate feature dimensions with equal distance

In [ ]:
num_total_dims = np.sum(regions_average_dims_array) # average each subject regions from Ncomp_SVHT_MG_DLD_approx
num_total_regions = 180 # 180 together
num_samples = 100

brain_region_dim_equal_div_array = np.linspace(num_total_regions, num_total_dims, num_samples)

brain_region_dim_scale_list = []
for i in brain_region_dim_equal_div_array:
  brain_region_dim_scale_list.append(num_total_dims/i)

# brain_region_dim_scale_list = brain_region_dim_scale_list[1:] # remove 180 
num_total_dims_list = []

# scale dims
for i_brain_region_dim_scale in brain_region_dim_scale_list:

  brain_region_dim_scale = i_brain_region_dim_scale 
  region_voxels_array = np.rint(copy.deepcopy(regions_average_dims_array)/brain_region_dim_scale).astype(int)
  num_total_dims = np.sum(region_voxels_array)
  # print("dims in total (with zero dims):", num_total_dims)
  # print("dims in total:", num_total_dims, ",", region_voxels_array)

  region_voxels_array[region_voxels_array == 0] = 1
  srm_dim_max = np.max(region_voxels_array)
  srm_dim_min = np.min(region_voxels_array)

  num_total_dims = np.sum(region_voxels_array)
  print("dims in total:", num_total_dims, ",", region_voxels_array)
  print("max:", srm_dim_max, ",min", srm_dim_min)

#### select the feature dimension with max number of effective regions 

In [ ]:
num_total_dims = np.sum(regions_average_dims_array) # average each subject regions from Ncomp_SVHT_MG_DLD_approx
num_total_regions = 180 # 180 together
num_samples = 100

brain_region_dim_equal_div_array = np.linspace(num_total_regions, num_total_dims, num_samples)

brain_region_dim_scale_list = []
for i in brain_region_dim_equal_div_array:
  brain_region_dim_scale_list.append(num_total_dims/i)
  
# index_max_num_significant_regions equals to 23, 1409
index_max_num_significant_regions = 23

# scale dims
print("brain_region_dim_scale:", brain_region_dim_scale_list[index_max_num_significant_regions])
brain_region_dim_scale = brain_region_dim_scale_list[index_max_num_significant_regions] 
region_voxels_array = np.rint(copy.deepcopy(regions_average_dims_array)/brain_region_dim_scale).astype(int)
num_total_dims = np.sum(region_voxels_array)
# print("dims in total (with zero dims):", num_total_dims)
# print("dims in total:", num_total_dims, ",", region_voxels_array)
region_voxels_array[region_voxels_array == 0] = 1
srm_dim_max = np.max(region_voxels_array)
srm_dim_min = np.min(region_voxels_array)

num_total_dims = np.sum(region_voxels_array)
print("dims in total:", num_total_dims)
# print("region_voxels_array:", region_voxels_array)
print("max:", srm_dim_max, ",min", srm_dim_min)


#### load and save "region_voxels_array"

##### save

In [ ]:
from scipy.io import savemat
import copy

print("region_voxels_array shape:", region_voxels_array.shape)

matdic = {"region_voxels_array": region_voxels_array}
project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/igeo_process"

file_dir = project_dir+"/process/depth/temp_data/"
savemat(file_dir + "vit_"+ str(num_total_dims) + "_dims" +"_region_voxels_array.mat", matdic)


##### load

In [15]:
from scipy.io import loadmat

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/igeo_process"
file_dir = project_dir+"/process/depth/temp_data/"
matdic = loadmat(file_dir + "vit_"+ str(num_total_dims) + "_dims" +"_region_voxels_array.mat")

region_voxels_array = np.squeeze(matdic["region_voxels_array"])
print("region_voxels_array shape =", region_voxels_array.shape)

srm_dim_max = np.max(region_voxels_array)
srm_dim_min = np.min(region_voxels_array)
print("max:", srm_dim_max, ",min", srm_dim_min)

region_voxels_array shape = (180,)
max: 36 ,min 1


## 3. split data into training and test

In [ ]:
# spliting into train and test data
train_data = []
test_data = []

for i_region in range(0, len(brain_region_name_list)):

  each_region_train_data = []
  each_region_test_data = []

  for i_sub in range(0, num_subs):

    temp_masked_epi_data = sub_masked_epi_data_list[i_sub][i_region]

    temp_train_epi_data = np.concatenate((temp_masked_epi_data[:,0:223*10], temp_masked_epi_data[:,223*13:223*14]), axis=1)

    temp_test_epi_data = temp_masked_epi_data[:,223*10:223*13]

    each_region_train_data.append(temp_train_epi_data)
    each_region_test_data.append(temp_test_epi_data)

  train_data.append(each_region_train_data)
  test_data.append(each_region_test_data)


In [ ]:
print('train_data shape:', np.asarray(train_data).shape)
print('train_data shape:', np.asarray(train_data[0][0]).shape)
print('test_data shape:', np.asarray(test_data).shape)
print('test_data shape:', np.asarray(test_data[0][0]).shape)

In [ ]:
srm_list = []
for i_region in range(0, len(brain_region_name_list)):

  temp_train_data = train_data[i_region]
  features = 10  # features
  n_iter = 200  # iterations of fitting

  # Create the SRM object
  srm = brainiak.funcalign.srm.SRM(n_iter=n_iter, features=features)

  # Fit the SRM data
  print(brain_region_name_list[i_region], ' Fitting...')
  srm.fit(temp_train_data)
  print(brain_region_name_list[i_region], ' SRM has been fit.')

  srm_list.append(srm)

## 4. train all data

### train the shared response model by regions

In [ ]:
# spliting into train and test data
train_data = []

for i_region in range(0, len(brain_region_name_list)):

  each_region_train_data = []

  for i_sub in range(0, num_subs):

    temp_masked_epi_data = sub_masked_epi_data_list[i_sub][i_region]

    each_region_train_data.append(temp_masked_epi_data)

  train_data.append(each_region_train_data)


In [ ]:
srm_list = []
for i_region in range(0, len(brain_region_name_list)):

  temp_train_data = train_data[i_region]
  n_iter = 200  # iterations of fitting

  features = region_voxels_array[i_region]
  # Create the SRM object
  srm = brainiak.funcalign.srm.SRM(n_iter=n_iter, features=features)

  # Fit the SRM data
  print(brain_region_name_list[i_region], ' Fitting...')
  srm.fit(temp_train_data)
  # print(brain_region_name_list[i_region], ' SRM has been fit.')

  srm_list.append(srm)

### save shared response

In [ ]:
from scipy.io import savemat
import copy

semgeo_shared_glm_list = []

for i_region in range(0, len(brain_region_name_list)):
  shared_response = copy.deepcopy(srm_list[i_region].s_)
  # print(brain_region_name_list[i_region], shared_response.shape)
  semgeo_shared_glm_list.append(shared_response)

print("num_total_dims:", num_total_dims)
matdic = {"semgeo_shared_glm_list":semgeo_shared_glm_list}
file_dir = project_dir+"/process/depth/temp_data/"
savemat(file_dir + "vit_"+ str(num_total_dims) + "_dims" +"_semgeo_shared_glm_list_optimal_hard_threshold.mat", matdic)
